In [1]:
import json
import os
import random
import sys
import pickle

In [2]:
import numpy as np
import pandas as pd

In [3]:
from PIL import Image
from numpy import asarray
import jax.numpy as jnp
import jax
from jax import jit
from jax import random

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
sys.path.insert(0,'/content/gdrive/MyDrive/Colab Notebooks/Capstone Project/Dependencies')

In [6]:
import transferlearningresnet as resnetModel

TEST DATA LOADING

In [7]:
path = 'gdrive/MyDrive/Colab Notebooks/Capstone Project/Polyvore_Data/'
json_file = os.path.join(path, 'Json Files','test_no_dup.json')

In [8]:
all_sets = json.load(open(json_file))

PARAMETERS LOADING

In [9]:
param_path = '/content/gdrive/MyDrive/Colab Notebooks/Capstone Project/params.p'

params = pickle.load( open( param_path, "rb" ) )
params.keys()

dict_keys(['linear', 'linear_1', 'linear_2', 'lstm/linear', 'lstm_1/linear', 'visual_semantic'])

In [ ]:
params['linear']['w'].shape, params['visual_semantic']['wv'].shape

((512, 512), (512, 512))

EMBEDDING GENERATION

In [ ]:
imgSize = (224,224,3)
model_resnet = resnetModel.getResNetModel(imgSize)
testData_path = 'gdrive/MyDrive/Colab Notebooks/Capstone Project/Polyvore_Data/Outfits'

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
def embeddingGenerator(image, img_size, model, embedding_params):
  Param_RNN, Param_VisualSemantic = embedding_params
  if image.mode != 'RGB':
    image = image.convert('RGB')
  img_resized = image.resize(img_size[:-1])
  img_numpy =  asarray(img_resized)
  image_feature = model.predict(np.array([img_numpy]))
  image_feature = image_feature.reshape([-1])

  image_embedding_RNN      =       jnp.dot(jnp.transpose(Param_RNN), image_feature)
  image_embedding_visualSemantic = jnp.dot(jnp.transpose(Param_VisualSemantic), image_feature)

  return image_embedding_RNN, image_embedding_visualSemantic


In [ ]:
def imageEmbedding(outfits, dataPath, img_size, model, embedding_params):
  image_names = []
  image_RNNs = []
  image_visualSemantics = []
  for outfit in outfits:
    set_id = outfit['set_id']
    outfit_path = os.path.join(dataPath, str(set_id))
    if os.path.exists(outfit_path):
      for image_info in outfit['items']:
        image_name = set_id + "_" + str(image_info["index"])
        image_path = os.path.join(outfit_path, str(image_info['index']) + '.jpg')
        image = Image.open(image_path)
        image_RNN, image_visualSemantic = embeddingGenerator(image, img_size, model, embedding_params)
        image_names.append(image_name)
        image_RNNs.append(image_RNN)
        image_visualSemantics.append(image_visualSemantic)
        
  return (image_names, np.array(image_RNNs), np.array(image_visualSemantics))


In [ ]:
test_features = imageEmbedding(all_sets, testData_path, imgSize, model_resnet, (params['linear']['w'], params['visual_semantic']['wv']))

In [ ]:
a,b,c = test_features

len(a), type(a), b.shape, c.shape

(14, list, (14, 512), (14, 512))

In [ ]:
import pickle

test_features_path = '/content/gdrive/MyDrive/Colab Notebooks/Capstone Project/test_features.p'

pickle.dump( test_features, open( test_features_path, "wb" ) )

CACHE

In [ ]:
# def imageEmbedding(outfits, dataPath, img_size, model, embedding_params):
#   test_features = {}
#   for outfit in outfits:
#     set_id = outfit['set_id']
#     outfit_path = os.path.join(dataPath, str(set_id))
#     if os.path.exists(outfit_path):
#       for image_info in outfit['items']:
#         image_name = set_id + "_" + str(image_info["index"])
#         image_path = os.path.join(outfit_path, str(image_info['index']) + '.jpg')
#         image = Image.open(image_path)
#         imageEmbeddings = embeddingGenerator(image, img_size, model, embedding_params)
#         test_features[image_name] = imageEmbeddings
        
#   return test_features


In [ ]:
# test_features.keys(), test_features['119314458_3'][0].shape, test_features['119314458_3'][1].shape, type(test_features['119314458_3'][1])